In [1]:
def read_file(file):
    import csv
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        header = next(reader)
        contacts = [[line[0], line[1]] for line in reader]
    return contacts

In [2]:
def read_template(message):
    from string import Template
    with open(message, 'r', encoding='utf-8') as f:
        content = f.read()
    return Template(content)

In [10]:
def send_messages(contact_file, message_file):
    
    recipients = read_file(contact_file)
    message_template = read_template(message_file)
    
    import smtplib
    from mail_data import EMAIL, PASS
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(EMAIL, PASS)
    
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders
    from PIL import Image
    from io import BytesIO

    for contact in recipients:
        name = contact[0]
        email = contact[1]

        message = MIMEMultipart()

        named_template = message_template.substitute(RECIPIENT_NAME=name.title())

        message['From'] = EMAIL
        message['To'] = email
        message['Subject'] = "Subject"
        message.attach(MIMEText(named_template, 'plain'))
        
        image = Image.open('img.jpeg')
        bytes = BytesIO()
        image.save(bytes, format='JPEG')
        att = MIMEBase('image', 'jpeg')
        att.set_payload(bytes.getvalue())
        encoders.encode_base64(att)
        att.add_header('Content-Disposition', 'attachment', filename='img.jpeg')
        
        message.attach(att)
        
        server.send_message(message)

        del message
    
    server.quit()


In [11]:
send_messages('contacts.csv', 'message.txt')

** Create a DB and insert dummy users **

In [17]:
import pymongo
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.email_app
collection=db.users

In [18]:
users = [{'username': 'admin', 'password': 'password'}, {'username': 'guest', 'password': '321321'}]
for user in users:
    collection.insert_one(user)